In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable


import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
class MARVEL_dataset(Dataset):
    def __init__(self, dat_file,train = True, transform = None):   
        self.root_dir = os.path.dirname(dat_file)
        datContent = [i.strip().split(',') for i in open(dat_file).readlines()]
        if train:
            csv_file = os.path.join(self.root_dir, "data_Train.csv")
        else:
            csv_file = os.path.join(self.root_dir, "data_Test.csv")
        with open(csv_file, "w") as f:
            writer = csv.writer(f,delimiter=',')
            writer.writerow(["counter", "set", "class", "label","location"])
            for line in datContent:
                if train and line[1]=='1':
                    if not(line[4] == '-'):
                        writer.writerows([line])  
                if not(train) and line[1] == '2':
                    if not(line[4]=='-'):
                        writer.writerows([line]) 
                
        self.MARVEL_datafile = pd.read_csv(csv_file)
        
        self.transform = transform
        
    def __len__(self):
        return len(self.MARVEL_datafile)
    
    def __getitem__(self,idx):
        img_name = self.MARVEL_datafile.iloc[idx,4]
        image = self.__loadfile(img_name)
        target = self.MARVEL_datafile.iloc[idx,2]
        if self.transform:
            image = Image.fromarray(image)
            sample = self.transform(image)
        else:
            sample = image
        return (sample,target)
    
    def __loadfile(self, data_file):
        image = io.imread(data_file)
        if len(image.shape)<3:
            image = np.stack((image,)*3, axis=-1)
        return image

In [ ]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)            
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])


In [ ]:
dat_file = "/home/rita/JupyterProjects/EYE-SEA/DataSets/marveldataset2016-master/FINAL.dat"
image_datasets = {'train': MARVEL_dataset(dat_file,train = True,transform=dataset_transform),'val': MARVEL_dataset(dat_file,train = False,transform=dataset_transform)}
print("Initializing Datasets and Dataloaders...")
# Create training and validation dataloaders
dataloaders = {'train': DataLoader(image_datasets['train'], batch_size=100, shuffle=True) , 'val': DataLoader(image_datasets['val'], batch_size=100, shuffle=True)}
print("Initializing Datasets and Dataloaders...")

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [ ]:
def train_model(model,criterion,optimizer,scheduler,num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch,num_epochs-1))
        print('-'*10)


        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()


            running_loss = 0.0
            running_corrects = 0

            for batch_id, (inputs, labels) in enumerate(dataloaders[phase]):
                print("labels {}".format(labels))
                #labels =torch.eye(NUM_CLASSES).index_select(dim=0, index=labels).long()
                inputs, labels = Variable(inputs), Variable(labels)
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    print(preds)
                    loss = criterion(torch.max(outputs).double(),labels.long())

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss : {:.4f}  Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))

            if phase == 'val' and epoch_acc>best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())


        print()

    time_elapsed = time.time() - since
    print('Training complete in  {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
    print('Best val Acc: {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
print("CONVNET AS FIXED FEATURES EXTRACTOR")
model_conv = torchvision.models.resnet18(pretrained=True)
#-----------------------This is the main thing to have a pretrained with fixed features -----------------
for param in model_conv.parameters():
    param.requires_grad = False
#--------------------------------------------------------------------------------------------------------

#this operation add the 2 new layers in the end
num_ftrs_conv = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs_conv,10)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler_conv = lr_scheduler.StepLR(optimizer_conv, step_size=7,gamma=0.1)

print("START TRAINING")
model_conv = train_model(model_conv,criterion,optimizer_conv,exp_lr_scheduler_conv,num_epochs=25)

visualize_model(model_conv)

plt.ioff()
plt.show()

In [ ]:
model_conv = torchvision.models.resnet18(pretrained=True)
res18_conv = nn.Sequential(*list(model_conv.children())[:-4])

res18_conv = res18_conv.to(device)
inputs, labels = next(iter(dataloaders['train']))
inputs, labels = Variable(inputs), Variable(labels)

inputs = inputs.to(device)
labels = labels.to(device)
print(inputs.shape)

outputs = res18_conv(inputs)
print(outputs.data.shape)


import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure(1)
image = inputs[1,:,:,:]
image = np.transpose(image[:,:,:].data.cpu().numpy(),(1,2,0))
print(image.shape)
plt.imshow(image.astype(np.float))
plt.show()
fig = plt.figure(2)
image = outputs[1,:,:,:]
image = np.transpose(image[0:3,:,:].data.cpu().numpy(),(1,2,0))
print(image.shape)
plt.imshow(image.astype(np.float))
plt.show()


In [ ]:
dataset_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])
batch_size = 100
image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
print("Initializing Datasets and Dataloaders...")

dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'] , batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
print("Initializing Datasets and Dataloaders...")

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10